In [9]:
from datetime import datetime
import calendar
import math

def calculate_precise_months(d1, d2):
    if d1 > d2:
        d1, d2 = d2, d1
    if d1.year == d2.year and d1.month == d2.month:
        days_in_month = calendar.monthrange(d1.year, d1.month)[1]
        return (d2 - d1).days / days_in_month
    else:
        days_in_start_month = calendar.monthrange(d1.year, d1.month)[1]
        start_fraction = (days_in_start_month - d1.day + 1) / days_in_start_month
        days_in_end_month = calendar.monthrange(d2.year, d2.month)[1]
        end_fraction = d2.day / days_in_end_month
        full_months = (d2.year - d1.year) * 12 + (d2.month - d1.month) - 1
        full_months = max(full_months, 0)
        return start_fraction + full_months + end_fraction

# Input handling
injection_dates = input("Enter the injection dates (comma-separated YYYY-MM-DD): ").split(',')
injection_prices = list(map(float, input("Enter the corresponding injection prices (comma-separated, in $/MMBtu): ").split(',')))
withdrawal_dates = input("Enter the withdrawal dates (comma-separated YYYY-MM-DD): ").split(',')
withdrawal_prices = list(map(float, input("Enter the corresponding withdrawal prices (comma-separated, in $/MMBtu): ").split(',')))

rate = float(input("Enter the rate at which the gas is injected/withdrawn (in million MMBtu): "))
injection_withdrawal_cost = float(input("Enter the injection/withdrawal cost (in $ per 1 million MMBtu): "))
storage_cost_rate = float(input("Enter the storage costs (Per month in $): "))
max_volume = float(input("Enter the maximum volume that can be stored (in million MMBtu): "))

# Convert dates
in_dates = [datetime.strptime(d.strip(), "%Y-%m-%d") for d in injection_dates]
out_dates = [datetime.strptime(d.strip(), "%Y-%m-%d") for d in withdrawal_dates]

# Ensure chronological order
all_dates = sorted(set(in_dates + out_dates))
volume = 0
buy_cost = 0
cash_in = 0

for current_date in all_dates:
    if current_date in in_dates:
        idx = in_dates.index(current_date)
        if volume + rate <= max_volume:
            volume += rate
            price = injection_prices[idx]
            buy_cost += rate * 1000000 * price + rate * injection_withdrawal_cost
            print(f"Injected gas on {current_date.date()} at price ${price}")
        else:
            print(f"Injection is not possible on {current_date.date()} due to insufficient space.")
    elif current_date in out_dates:
        idx = out_dates.index(current_date)
        if volume >= rate:
            volume -= rate
            price = withdrawal_prices[idx]
            cash_in += rate * 1000000 * price - rate * injection_withdrawal_cost
            print(f"Withdrawn gas on {current_date.date()} at price ${price}")
        else:
            print(f"Withdrawal is not possible on {current_date.date()} due to insufficient stored volume.")

# Calculate precise storage cost
total_months = calculate_precise_months(min(in_dates), max(out_dates))
storage_cost = total_months * storage_cost_rate

contract_value = cash_in - buy_cost - storage_cost

print(f"\nThe value of the contract is: ${contract_value:.2f}")

Enter the injection dates (comma-separated YYYY-MM-DD):  2024-05-01,2024-06-01
Enter the corresponding injection prices (comma-separated, in $/MMBtu):  2,2.9
Enter the withdrawal dates (comma-separated YYYY-MM-DD):  2024-07-01,2024-09-01
Enter the corresponding withdrawal prices (comma-separated, in $/MMBtu):  2.9,3.5
Enter the rate at which the gas is injected/withdrawn (in million MMBtu):  1
Enter the injection/withdrawal cost (in $ per 1 million MMBtu):  10000
Enter the storage costs (Per month in $):  100000
Enter the maximum volume that can be stored (in million MMBtu):  2


Injected gas on 2024-05-01 at price $2.0
Injected gas on 2024-06-01 at price $2.9
Withdrawn gas on 2024-07-01 at price $2.9
Withdrawn gas on 2024-09-01 at price $3.5

The value of the contract is: $1056666.67
